## Tratamento dos dados via Open Street Map

#### Carregando a base de dados:


In [2]:
import pandas as pd

estabelecimentos = pd.read_csv("../dados/20250401_atividade_economica.csv", sep=";")

#### Filtrando bares/restaurantes e adaptando nome do logradouro para pesquisa no OSM

In [ ]:
bares_restaurantes = estabelecimentos[estabelecimentos['DESCRICAO_CNAE_PRINCIPAL'].str.contains('restaurante', case=False) | estabelecimentos['DESCRICAO_CNAE_PRINCIPAL'].str.contains('bar',case=False)]

bares_restaurantes['DESC_LOGRADOURO'] = bares_restaurantes['DESC_LOGRADOURO'].replace({'AVE':'AVENIDA',
                                                                                       'PCA':'PRACA',
                                                                                       'ALA':'ALAMEDA',
                                                                                       'ROD':'RODOVIA',
                                                                                       'BEC':'BECO',
                                                                                       'TRV':'TREVO',
                                                                                       'LRG':'LARGO',
                                                                                       'EST':'ESTRADA',
                                                                                       'VDP':''})

bares_restaurantes['DESC_LOGRADOURO'].unique()


C:\Users\grpas\AppData\Local\Temp\ipykernel_11432\1260400138.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bares_restaurantes['DESC_LOGRADOURO'] = bares_restaurantes['DESC_LOGRADOURO'].replace({'AVE':'AVENIDA',


array(['RUA', 'AVENIDA', 'ALAMEDA', 'RODOVIA', 'PRACA', 'BECO', 'TREVO',
       'LARGO', 'ESTRADA', 'VIA', ''], dtype=object)

#### Adicionando coordenadas geográficas pelo OSM

In [ ]:
from OSMPythonTools.nominatim import Nominatim


nominatim = Nominatim()

def get_lat_lon(address):
    #EX: "221B Baker Street, London"

    result = nominatim.query(address)

    # Depending on the result, OSMPythonTools may return a single object or a list of objects.
    if isinstance(result, list):
        # If multiple results are returned, pick the first one (or choose based on your criteria)
        osm_object = result[0]
    else:
        osm_object = result
    return osm_object.toJSON()[0]['lat'], osm_object.toJSON()[0]['lon'] 



In [ ]:

for index, row in bares_restaurantes.iterrows():
    endereco = str(int(row['NUMERO_IMOVEL']))+ ", " +row['DESC_LOGRADOURO'] +" "+ row['NOME_LOGRADOURO'] + ", " + row['NOME_BAIRRO'] + ", Belo Horizonte, MG, Brasil"
    try:

        lat, lon = get_lat_lon(endereco)
        bares_restaurantes.at[index, 'LATITUDE'] = lat
        bares_restaurantes.at[index, 'LONGITUDE'] = lon
    except Exception :
        print(f'Erro processando "{endereco}"')


NameError: name 'bares_restaurantes' is not defined

#### Salvando em arquivo CSV

In [ ]:
bares_restaurantes.to_csv('../dados/bares_restaurantes.csv', sep=';', index=False)
bares_restaurantes

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA,LATITUDE,LONGITUDE
17,1023,5611201,RESTAURANTES E SIMILARES,5611201,01-07-1993,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,82.0,S,S,...,DESEMBARGADOR REIS ALVES,90.0,"ANDAR: 2,",BAIRRO DAS INDUSTRIAS I,APARECIDA MARIA DE SOUZA,NaN,7.110299e+13,POINT (604468.46 7792708.63),-19.9589334,-44.0017603
19,1025,5611201,RESTAURANTES E SIMILARES,5611201,15-10-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,125.0,N,N,...,LUIZ PONGELUPE,290.0,C,CARDOSO,LUCAS SILVA DE ARAUJO LTDA,PIZZARIA E CHURRASCARIA VARANDA,7.116489e+13,POINT (603585.25 7787403.83),-20.0062900,-44.0100157
29,1056,5611201,RESTAURANTES E SIMILARES,5611201,02-05-1994,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,42.0,S,N,...,RESSACA,118.0,LOJA 04,PADRE EUSTAQUIO,PAULO EMILIO COELHO,RABBIT BURGER,9.750650e+13,POINT (606050.57 7796975.21),NaN,NaN
172,1071,5611201,RESTAURANTES E SIMILARES,"5611201, 5611203",24-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,74.0,S,N,...,DESEMBARGADOR RIBEIRO DA LUZ,135.0,NaN,BARREIRO,GARAPAO BAR E RESTAURANTE LTDA,TATU REI DO ANGU A BAHIANA,7.139363e+13,POINT (603011.98 7791138.07),-19.9738684,-44.0169993
196,1095,4614100,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,"4614100, 4744001, 8599604",01-06-1994,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,36.0,S,N,...,ELIAS ANTONIO ISSA,288.0,NaN,LETICIA,ROD BITS COMERCIO E REPRESENTACAO LTDA,NaN,7.479400e+10,POINT (607384.66 7809478.88),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,482862,5611201,RESTAURANTES E SIMILARES,5611201,27-08-2024,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,CIPRIANO DE CARVALHO,526.0,NaN,CINQUENTENARIO,RESTAURANTE PANELA VELHA MINEIRA LTDA,NaN,5.703275e+13,POINT (606204.32 7793152.82),-19.9567158,-43.9845731
20635,482871,5611201,RESTAURANTES E SIMILARES,"5611201, 5611203, 5611204",27-08-2024,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,90.0,S,N,...,DOUTOR CRISTIANO GUIMARAES,280.0,LOJA: C,PLANALTO,BAR DO TICO LTDA,BAR DO TICO,5.703342e+13,POINT (609752.43 7807660.93),NaN,NaN
20735,479267,5611201,RESTAURANTES E SIMILARES,"5611201, 5611203",08-08-2024,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,59.0,S,N,...,PAULO AFONSO,308.0,LOJA: 2,SANTO ANTONIO,REEN BURGUER BH LTDA,REEN BURGUUER BH,5.641444e+13,POINT (610306.55 7794870.87),-19.9374625,-43.9456292
20875,481558,5611205,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...,5611205,21-08-2024,EMPRESA SIMPLES DE INOVAÇÃO - INOVA SIMPLES,MICROEMPRESA - ME,10.0,N,N,...,MEXICO,151.0,NaN,COPACABANA,BOTECO COMERCIAL DISTRIBUIDORA INOVA SIMPLES (...,NaN,5.694406e+13,POINT (605401.36 7806667.07),-19.8331209,-43.9934277
